In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
gamedata = pd.read_csv("/kaggle/input/popular-video-games-1980-2023/games.csv")

In [ ]:
gamedata.shape

In [ ]:
gamedata.head(10)

In [ ]:
gamedata.columns

**About the data**
* *Unnamed : 0* : unnecessary since it's the same with index 
* *Title* : Title of the Game.
* *Release Date* : Date of release of the game's first version
* *Team* : Game developer team
* *Rating* : Average rating
* *Times Listed* : Number of users who listed this game
* *Number of Reviews* : Number of reviews received from the users
* *Genres* : All genres pertaining to a specified game
* *Summary* : Summary provided by the team
* *Reviews* : User reviews
* *Plays* : Number of users that have played the game before
* *Playing* : Number of current users who are playing the game.
* *Backlogs* : Number of users who have access but haven't started with the game yet
* *Wishlist* : Number of users who wish to play the game

*we observe that the columns 'Unnamed :0', 'Reviews', 'Times Listed'(same as number of reviews) and 'Summary' is unnecessary for our analysis so we'll drop them for cleaning the data.*

In [ ]:
gamedata.info()

In [ ]:
#gamedata['Team'].isnull().any()
columns_with_null = gamedata[gamedata['Team'].isnull()]

# Display the names of columns with null values
columns_with_null

we'll not contain this game for analysis...

In [ ]:
columns_with_null = gamedata[gamedata['Rating'].isnull()]

# Display the names of columns with null values
columns_with_null

Observe that all these games were just released or not released yet when this dataset was created. So we'll drop these games from our analysis too. So, let's create another dataframe that contains these values:

In [ ]:
dfdel = columns_with_null
dfdel.info()


# **Cleaning the data**

In [ ]:
#creating new dataframe without 'Unnamed:0', 'Reviews' , 'Times Listed' and 'Summary'
df = gamedata[['Title', 'Release Date', 'Team', 'Rating',
       'Number of Reviews', 'Genres','Plays', 'Playing',
       'Backlogs', 'Wishlist']]
df

In [ ]:
#Checking if there's any duplicates
duplicate = df[df.duplicated()]

duplicate

In [ ]:
#drop duplicates
df2 = df.drop_duplicates(keep='first')
df2.shape

In [ ]:
#checking if we have any NaN values
df2.isna().any()

We have some NaN values in 'Team' and 'Rating'. Let's check what these values are:

In [ ]:
# Find and display the null values in a specific column
nulls_team = df2[df2['Team'].isnull()]
nulls_rating = df2[df2['Rating'].isnull()]

# Display the rows where 'column_name' has null values
print(nulls_team,nulls_rating)


We see that non-rated games are either not released yet or just released when this dataset created. We will separate these values from our dataset.
Only one game doesn't have a team, we will also drop this game from our dataset since 'team' column is important for our analysis.

In [ ]:
nulls_rating.shape

we have 14 rows to delete, so we should have 1120-14 = 1106 row at the end of cleaning

In [ ]:
# Using the dropna method to remove the rows with NaN value(s) and resetting the index
df2 = df2.dropna()
df2 = df2.reset_index(drop=True)
df2

In [ ]:
#Checking if there is still any 'releases on TBD' left
filtered_df = df2[df2['Release Date'] == 'releases on TBD']

filtered_df

I checked Deltarune's release date and found out that it is October 31,2018 so we will add its release date. Elden Ring: Shadow of the Erdtree still not released yet so we will drop it from our dataset.

In [ ]:
#Changing the game Deltarune's release date to 'Oct 31,2018' instead of 'releases on TBD'
df2.at[420, 'Release Date'] = 'Oct 31, 2018'
df2.loc[420]

In [ ]:
#Drop the game 'Elden Ring: Shadow of the Erdtree'.
df2 = df2.drop(index=869)
df2 = df2.reset_index(drop=True)
df2

In [ ]:
df2.info()

we only have 'Rating' as a float type object so we will convert other numerical values to float aswell.

In [ ]:
columns_to_convert = ['Number of Reviews','Plays','Playing','Backlogs','Wishlist']
def convert_k_to_int(x):
    if 'K' in x:
        return int(float(x.replace("K", "")) * 1000)
    else:
        return int(x)

# Apply the custom function to the selected columns
df2[columns_to_convert] = df2[columns_to_convert].applymap(convert_k_to_int)
df2[columns_to_convert]

We also need to convert Release Date to a numerical value:

In [ ]:
# Convert the 'Release Date' column to datetime
df2['Release Date'] = pd.to_datetime(df2['Release Date'])

# Extract numerical components
df2['Year'] = df2['Release Date'].dt.year
df2['Month'] = df2['Release Date'].dt.month
df2['Day'] = df2['Release Date'].dt.day

# Display the DataFrame with numerical date components
df2


In [ ]:
q10 = df2['Plays'].quantile(0.1)
q10

For a better analysis, it's better if we not include the games with total number of plays less than 10% quantile. We'll use it for some parts.

Also, we'll create another dataframe as more filtered version of this dataframe:

In [ ]:
df3 = df2
for col in ["Number of Reviews", "Plays", "Playing", "Backlogs", "Wishlist"]:
    col_data = df3[col]
    q1, q3 = col_data.quantile(0.25), col_data.quantile(0.75)
    iqr = q3 - q1 
    low_limit = q1 - 1.5 * iqr
    up_limit = q3 + 1.5 * iqr

    df3 = df3.loc[(df3[col] > low_limit) & (df3[col] < up_limit)]
df3

this dataframe is filtered for better outcomes, we will use this dataframe for some of our questions.

**1) Hangi oyuna en fazla review yapılmış?**

In [ ]:
df2['Number of Reviews'].idxmax()

In [ ]:
df2.loc[2]

**2) Oyunların Rating dağılımı ve grafiği nedir? (%10 quantile dahil ederek ve etmeyerek)**

In [ ]:
df['Rating'].value_counts()

In [ ]:
sns.set(style="darkgrid")
sns.histplot(data=df2, x="Rating", kde=True)
plt.show()

Now let's look with dropping the games with number of playing less than quantile10% :

In [ ]:
dfq10 = df2[df2['Plays'] >= q10]
dfq10

In [ ]:
sns.set(style="darkgrid")
sns.histplot(data=dfq10, x="Rating", kde=True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(x="Rating", y="Plays", data=dfq10)
plt.show()

In [ ]:
df2.describe()

In [ ]:
dfq10.describe()

**3) 4'ün üstünde oylama alan kaç oyun vardır?**

In [ ]:
df2[df2['Rating'] >= 4].count()

In [ ]:
dfq10[dfq10['Rating'] >= 4].count()

**4) En çok güncel olarak hangi oyun oynanmaktadır?**

In [ ]:
df2.loc[df2['Playing'].idxmax()]

**5) Oyunların yayınlanma yılına göre ortalama Rating ve Plays dağılımı**

In [ ]:
df2['Release Year'] = df2['Release Date'].dt.year

# Group by 'Release Year' and calculate the mean of 'Rating'
mean_ratings_by_year = df2.groupby('Release Year')['Rating'].mean()

plt.figure(figsize=(10, 5))
sns.barplot(x=mean_ratings_by_year.index, y=mean_ratings_by_year.values)
plt.title('Mean Ratings of Games by Release Year (Seaborn Bar Chart)')
plt.xlabel('Release Year')
plt.ylabel('Mean Ratings')
plt.grid(axis='y')
plt.xticks(rotation=45)  # Rotate x-axis labels as needed
plt.tight_layout()
plt.show()

**Oyunların yayınlanma yılına göre Plays dağılımı**

In [ ]:
# Group by 'Release Year' and calculate the mean of 'Rating'
mean_plays_by_year = df2.groupby('Release Year')['Plays'].mean()

plt.figure(figsize=(10, 5))
sns.barplot(x=mean_plays_by_year.index, y=mean_plays_by_year.values)
plt.title('Mean Number of Plays of Games by Release Year (Seaborn Bar Chart)')
plt.xlabel('Release Year')
plt.ylabel('Mean Number of Plays')
plt.grid(axis='y')
plt.xticks(rotation=45)  # Rotate x-axis labels as needed
plt.tight_layout()
plt.show()

In [ ]:
games_1985 = df2[df2['Release Year'] == 1985]
games_1985

**6) Oyunların çıkış yılına göre toplam ne kadar oynanmaya sahip olduğunun ve hangi yıl ne kadar oyun çıktığının grafikleri**

In [ ]:
sum_plays_by_year = df2.groupby('Release Year')['Plays'].sum()

plt.figure(figsize=(10, 5))
sns.barplot(x=sum_plays_by_year.index, y=sum_plays_by_year.values)
plt.title('Number of Plays of Games by Release Year (Seaborn Bar Chart)')
plt.xlabel('Release Year')
plt.ylabel('Mean Number of Plays')
plt.grid(axis='y')
plt.xticks(rotation=45)  # Rotate x-axis labels as needed
plt.tight_layout()
plt.show()

In [ ]:
noG_byyear = df2.groupby('Release Year').size().reset_index(name='Number of Games')
plt.figure(figsize=(10, 5))
plt.bar(noG_byyear['Release Year'], noG_byyear['Number of Games'])
plt.title('Number of Games Released per Year')
plt.xlabel('Release Year')
plt.ylabel('Number of Games')
plt.grid(axis='y')
plt.xticks(noG_byyear['Release Year'])
plt.xticks(rotation=45)
plt.show()

**7) Hangi oyun en çok backlogs/plays oranına sahiptir?**

In [ ]:
b_p_ratio = dfq10['Backlogs'] / dfq10['Plays']
b_p_ratio.max()

In [ ]:
dfq10.loc[b_p_ratio.idxmax()]

It might be the case that the game was in sale for a cheap price or free so people usually just added it to their library and not played it in general.

**8) Güncel olarak ortalama ne kadar oyun oynanmaktadır ve oynanmıştır?**

In [ ]:
df2['Playing'].mean()

In [ ]:
df2['Plays'].mean()

**9) En yüksek oynanmaya sahip oyunun ratingi nedir?**

In [ ]:
df2.loc[df2['Plays'].idxmax()]['Rating']

**10) Nintendo tarafından yapılmış oyunların rating ortalaması**

In [ ]:
nintendo = df2[df2['Team'].astype(str).apply(lambda x: 'Nintendo' in x)]
nintendo

In [ ]:
nintendo['Rating'].mean()

**11) En yüksek ratinge sahip oyunlar**

In [ ]:
df2.groupby('Title')['Rating'].max().sort_values(ascending=False).head(20)

**12) Oynanma sayısı ile backlog sayısı kıyaslama**

In [ ]:
sns.jointplot(x = "Plays", y = "Backlogs", data=df2, kind="hex")

In [ ]:
df2['Plays'].describe()

In [ ]:
df2['Backlogs'].describe()

In [ ]:
df75p = df2[df2['Plays'] >= df2['Plays'].quantile(0.75)]
df75m = df2[df2['Plays'] < df2['Plays'].quantile(0.75)]
df90p = df2[df2['Plays'] >= df2['Plays'].quantile(0.9)]
df75p

In [ ]:
sns.jointplot(x = "Plays", y = "Backlogs", data=df75p, kind="hex")

In [ ]:
sns.jointplot(x = "Plays", y = "Backlogs", data=df90p, kind="hex")

In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(x="Plays", y="Backlogs", data=df90p)
plt.show()

In [ ]:
df75m

In [ ]:
df90p

**13) En çok oynanan 10 oyunun wishlist grafiği** 

In [ ]:
#p= df2.groupby(['Title','Wishlist'])['Plays'].max().sort_values(ascending=False).head(10)
p = df2.sort_values(by='Plays', ascending=False).head(10)
p

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(p['Title'], p['Wishlist'])
plt.title('Wishlist Numbers of Top 10 Played Games')
plt.xlabel('Game')
plt.ylabel('Wishlist Numbers')
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for readability
plt.grid(axis='y')

plt.show()

**14) Hangi ayda ne kadar oyun çıkmış?**

In [ ]:
games_per_month = df2['Month'].value_counts().sort_index()
games_per_month.plot(kind='bar')

**15) En çok tercih edilen Janralar ve grafiği**

In [ ]:
df2['Genres'] = df2['Genres'].str.replace("[", "").str.replace("]", "").str.replace("'", "")

# Create a new dataframe with one-hot encoded genres
genres_df = df2['Genres'].str.split(', ', expand=True)

# Sum up the number of games in each genre
genre_counts = genres_df.stack().value_counts()

# Print the genres and their counts
print(genre_counts)

In [ ]:
sns.set_style('whitegrid')
ax = sns.barplot(x=genre_counts.index, y=genre_counts.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Genre Counts')
plt.xlabel('Genre')
plt.ylabel('Count')
plt.show()

**16) Veri kümesinde kaç oyun şirketi bulunmaktadır ve kaç oyunları vardır?**

In [ ]:
df2['Team'] = df2['Team'].str.replace("[", "").str.replace("]", "").str.replace("'", "")
teams = df2['Team'].str.split(', ', expand=True)
team_counts = teams.stack().value_counts()
team_counts

**17) En çok hangi takımın oyunları wishliste alınmıştır?**

In [ ]:
team_wishlist = df2.groupby('Team')['Wishlist'].sum()
team_name = team_wishlist.idxmax()
wishlist_count = team_wishlist.max()
print(f"The team '{team_name}' has the highest wishlist count of {wishlist_count}.")

**18) Ortalamadan fazla review sayısı olan oyunların oyun türü dağılımı nasıldır?**

In [ ]:
df2['Number of Reviews'].describe()

In [ ]:
num_rv = df2[df2['Number of Reviews']> 447]
numrv_genres = num_rv['Genres'].str.split(', ', expand=True)
numrv_count = numrv_genres.stack().value_counts()
numrv_count

In [ ]:
sns.set_style('whitegrid')
bx = sns.barplot(x=numrv_count.index, y=numrv_count.values)
bx.set_xticklabels(bx.get_xticklabels(), rotation=45, ha='right')
plt.title('Genre Counts')
plt.xlabel('Genre')
plt.ylabel('Count')
plt.show()

**19) Playing/Plays oranına göre en yüksek orana sahip ilk 10 oyun nedir?**

In [ ]:
df2['Play Ratio'] = df2['Playing']/df2['Plays']
df2.sort_values(by = 'Play Ratio',ascending=False).head(10)

In [ ]:
df3['Play Ratio'] = df3['Playing']/df3['Plays']
df3.sort_values(by = 'Play Ratio',ascending=False).head(10)

In [ ]:
dfq10['Play Ratio'] = dfq10['Playing']/dfq10['Plays']
dfq10.sort_values(by = 'Play Ratio',ascending=False).head(10)

**20) En çok backloga alınmış 50 oyunun Rating ortalamaları nedir?**

In [ ]:
game50 = df2.sort_values(by = 'Backlogs',ascending=False).head(30).reset_index()
game50['Rating'].mean()

In [ ]:
game50

# Linear Regression on Rating of Games

We will calculate the 'ages' of the games in 1 January 2023 to use how old are the games for prediction. We will use the filtered df3 dataframe.

In [ ]:
# Calculate the age of the games based on '2023-01-01' in days
target_date = pd.to_datetime('2023-01-01')
df3['Age'] = (target_date - df3['Release Date']).dt.days
df3

We'll also use the genre column for whether a game is indie or not.

In [ ]:
df3['Indie'] = df['Genres'].str.contains('Indie').astype(int)
df3.reset_index()

Taking only the columns that we need and filtering the games that released in 2023:

In [ ]:
ml_cols = df3[['Number of Reviews', 'Plays', 'Playing', 'Backlogs' , 'Wishlist' ,'Age' , 'Indie' ]]
ml_cols.drop(ml_cols[ml_cols['Age'] < 0].index, inplace=True)
ml_cols.sort_values(by = 'Age', ascending=True)

Fitting our values to smaller size:

In [ ]:
mlfit = ml_cols[['Number of Reviews', 'Plays', 'Playing', 'Backlogs' , 'Wishlist' ,'Age' ]] / 1000
mlfit['Indie'] = df3['Indie']
mlfit

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
X = mlfit
df3.drop(df3[df3['Age'] < 0].index, inplace=True)
y = df3['Rating']
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12)
X_train

In [ ]:
X_test

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
reg.coef_ 

In [ ]:
y_pred = reg.predict(X_test)
y_pred.round(1)

In [ ]:
y_test

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Y Test Values")
plt.ylabel("Y Pred Values")

plt.grid()

In [ ]:
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
print("Mean Squared Error: ", metrics.mean_squared_error(y_test, y_pred))

In [ ]:
print("Root Mean Squared Error: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
coeffs = pd.DataFrame(reg.coef_, X.columns)
coeffs.columns = ["Coefficient"]
coeffs

In [ ]:
reg.intercept_

Let's create 10 artificial datas and test it with our model :

In [ ]:
t10 = pd.DataFrame(index=range(10), columns=range(6))
t10.iloc[0, :] = [405, 887, 33, 17, 32, 1000]
t10.iloc[1, :] = [1500, 6875, 234, 811, 888, 5467]
t10.iloc[2, :] = [63, 778, 77, 18, 12, 1300]
t10.iloc[3, :] = [1070, 8473, 232, 2115, 336, 6789]
t10.iloc[4, :] = [266, 3220, 120, 32, 128, 35]
t10.iloc[5, :] = [367, 3786, 121, 39, 233, 90]
t10.iloc[6, :] = [1124, 4411, 223, 462, 424, 3366]
t10.iloc[7, :] = [899, 9807, 158, 532, 54, 14897]
t10.iloc[8,:] = [672 , 2768 ,59 ,60 ,123 ,626]
t10.iloc[9,:] = [1498 ,11278 ,326 ,1567 ,667,9876]
t10.rename(columns={0: 'Number of Reviews', 1: 'Plays', 2: 'Playing', 3: 'Backlogs', 4: 'Wishlist', 5: 'Age'}, inplace=True)
t10

In [ ]:
t10 = t10[['Number of Reviews', 'Plays', 'Playing', 'Backlogs' , 'Wishlist' ,'Age' ]] / 1000
t10.insert(6, 'Indie', [1, 0, 1, 0, 1, 1, 0, 1, 0, 0])
t10

In [ ]:
y10_pred = reg.predict(t10)
y10_pred.round(1)